## BEFORE YOU BEGIN

1. create a directory called 'raw_data' in the directory containing this notebook
2. add the data files to in `./raw_data` with the name "RAW_<type_of_data>.csv"
  - recipes: `RAW_recipes.csv`
    - https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions/data?select=RAW_recipes.csv
  - interactions: `RAW_interactions.csv`
    - https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions/data?select=RAW_interactions.csv
  - ingredients: `RAW_ingredients.csv`
    - https://www.kaggle.com/datasets/thedevastator/the-nutritional-content-of-food-a-comprehensive
  - allergens: `RAW_allergens.csv`
    - https://www.kaggle.com/datasets/boltcutters/food-allergens-and-allergies 
3. create a directory called 'cleaned_data' in the directory containing this notebook, this is where the output csvs will end up

In [368]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
import ast
import re

recipes = pd.read_csv('./raw_data/RAW_recipes.csv')
recipes['ingredients'] = recipes['ingredients'].apply(ast.literal_eval)

## Cleaning Ingredients

In [369]:
ingredients = pd.read_csv('./raw_data/RAW_ingredients.csv')
ingredients['Shrt_Desc'] = ingredients['Shrt_Desc'].str.lower().apply(lambda x: re.sub(r'\s*,\s*', ' ', x))
basics = {"onion": "onions", "apples": "gala apples", "cream": "whipping cream", "apple": "gala apple", "nuts": "mixed nuts", "whipping cream": "heavy whipping cream", "strawberry": "strawberries", "oil": "canola oil", "milk": "whole milk", "yogurt": "yogurt plain", "flour": "all-purpose flour", "sugar": "granulated sugar", "pepper": "black pepper", "eggs": "whole egg", "egg": "whole egg", "butter": "unsalted butter", "tomato": "red tomato", "tomatoes": "red tomatoes"}

In [370]:
ingredients

,index,NDB_No,Shrt_Desc,Water_(g),Energ_Kcal,Protein_(g),Lipid_Tot_(g),Ash_(g),Carbohydrt_(g),Fiber_TD_(g),...,Vit_K_(µg),FA_Sat_(g),FA_Mono_(g),FA_Poly_(g),Cholestrl_(mg),GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,Refuse_Pct
0,0,1001,butter with salt,15.87,717,0.85,81.11,2.11,0.06,0.0,...,7.0,51.368,21.021,3.043,215.0,5.00,"1 pat, (1"" sq, 1/3"" high)",14.2,1 tbsp,0.0
1,1,1002,butter whipped w/ salt,16.72,718,0.49,78.30,1.62,2.87,0.0,...,4.6,45.390,19.874,3.331,225.0,3.80,"1 pat, (1"" sq, 1/3"" high)",9.4,1 tbsp,0.0
2,2,1003,butter oil anhydrous,0.24,876,0.28,99.48,0.00,0.00,0.0,...,8.6,61.924,28.732,3.694,256.0,12.80,1 tbsp,205.0,1 cup,0.0
3,3,1004,cheese blue,42.41,353,21.40,28.74,5.11,2.34,0.0,...,2.4,18.669,7.778,0.800,75.0,28.35,1 oz,17.0,1 cubic inch,0.0
4,4,1005,cheese brick,41.11,371,23.24,29.68,3.18,2.79,0.0,...,2.5,18.764,8.598,0.784,94.0,132.00,"1 cup, diced",113.0,"1 cup, shredded",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8785,8785,83110,mackerel salted,43.00,305,18.50,25.10,13.40,0.00,0.0,...,7.8,7.148,8.320,6.210,95.0,80.00,"1 piece, (5-1/2"" x 1-1/2"" x 1/2"")",17.0,"1 cubic inch, boneless",0.0
8786,8786,90240,scallop (bay&sea) ckd stmd,70.25,111,20.54,0.84,2.97,5.41,0.0,...,0.0,0.218,0.082,0.222,41.0,85.00,3 oz,NaN,NaN,0.0
8787,8787,90480,syrup cane,26.00,269,0.00,0.00,0.86,73.14,0.0,...,0.0,0.000,0.000,0.000,0.0,21.00,1 serving,NaN,NaN,0.0
8788,8788,90560,snail raw,79.20,90,16.10,1.40,1.30,2.00,0.0,...,0.1,0.361,0.259,0.252,50.0,85.00,3 oz,NaN,NaN,0.0


In [371]:
remove_keywords = ["sugar sub", "entree", "ckd", "infant formula", "inf formula", "child formula", "meat only", "skin only", "applebee's", "t.g.i friday's", "restaurant", "denny's", "ckd", "babyfood", "fast foods", "burger king", "mcdonald's", "wendy's", "pizza hut", "kfc", "taco bell", "subway", "domino's", "papa john's", "little caesars", "chick-fil-a", "popeyes", "olive garden", "on the border", "carraba's", "cracker barrel"]
replace_raw = {'pork cured bacon': "bacon", 'commly prep': "", 'no chol': "", 'unenr': "", 'raw': "", 'frsh': "fresh", 'frz': "frozen", 'crm': "cream", 'pnut': "peanut", 'pln': "plain", 'whl': "whole", 'dk': "dark", 'broilers or fryers': "", 'drsng': "dressing", 'spread': "spread", 'flr': "flour", 'bf': "beef" ,'w/': "with", 'wo/': "without", 'hvy': "heavy", 'lt': "light", 'juc': "juice", 'wo/salt': "unsalted", 'wo/ salt': "unsalted", 'without salt': "unsalted", "salad or cooking": "", 'yel': "yellow", 'sau': "sauce", 'veg': "vegetable"}

mask = ingredients['Shrt_Desc'].str.contains('|'.join(remove_keywords))
ingredients = ingredients[~mask]

replace = {f'^{key}\\b': value + " " for key, value in replace_raw.items()}
replace.update({f'\\b{key}$': " " + value for key, value in replace_raw.items()})
replace.update({f'\\b{key}\\b': " " + value + " " for key, value in replace_raw.items()})
replace.update({'\\bmilkfat\\b.*$': ""})
replace.update({'\\((.*?)\\)': ""})

ingredients.reset_index(drop=True, inplace=True)
ingredients["Cleaned_Desc"] = ingredients["Shrt_Desc"].replace(replace, regex=True).replace(r'\s+', ' ', regex=True).str.strip()
ingredient_names = ingredients['Cleaned_Desc'].unique()

ingredient_names

C:\Users\aprin\AppData\Local\Temp\ipykernel_14528\1717127808.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingredients["Cleaned_Desc"] = ingredients["Shrt_Desc"].replace(replace, regex=True).replace(r'\s+', ' ', regex=True).str.strip()


array(['butter with salt', 'butter whipped w/ salt',
       'butter oil anhydrous', ..., 'syrup cane', 'snail', 'turtle green'],
      dtype=object)

In [372]:
def matchWord(target, tomatch):
  l = list(map(lambda x : x[0], process.extractBests(target, tomatch, scorer=fuzz.token_sort_ratio, score_cutoff=50, limit=20)))
  return process.extractOne(target, l, scorer=fuzz.token_set_ratio)

#### Get recipes that match cleaned ingredients

In [373]:
valid = dict()
bad_match = dict()
no_match = []
for _, recipe in recipes.loc[:20000].iterrows():
  skip = False
  ingreds_in_recipe = dict()
  for ingred in recipe["ingredients"]:
    to_match = ingred
    if ingred in basics.keys():
      to_match = basics[ingred]

    res = matchWord(to_match, ingredient_names)
    if res is None:
      no_match.append(to_match)
      skip = True
      break
    match, percent = res
    ingreds_in_recipe[to_match] = match
    if percent < 90:
      bad_match[to_match] = (match, percent)
      skip = True
      break
  if not skip:
    # print(ingreds_in_recipe)
    valid[recipe['name']] = ingreds_in_recipe
    skip = False

## Get needed ingredients

In [374]:
print("Valid recipes: ", len(valid))
unique_ingreds = list(set(ingred for recipe in valid.values() for ingred in recipe.values()))
print("Unique ingredients in recipes: ", len(unique_ingreds))

mask = ingredients['Cleaned_Desc'].isin(unique_ingreds)
unneeded_cols = ["Shrt_Desc", "Refuse_Pct", "index"]
ingredients_csv = ingredients[mask].drop(columns=unneeded_cols).drop_duplicates(subset=['Cleaned_Desc'], keep='first').reset_index(drop=True)
ingredients_csv


Valid recipes:  1411
Unique ingredients in recipes:  544


,NDB_No,Water_(g),Energ_Kcal,Protein_(g),Lipid_Tot_(g),Ash_(g),Carbohydrt_(g),Fiber_TD_(g),Sugar_Tot_(g),Calcium_(mg),...,Vit_K_(µg),FA_Sat_(g),FA_Mono_(g),FA_Poly_(g),Cholestrl_(mg),GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,Cleaned_Desc
0,1004,42.41,353,21.40,28.74,5.11,2.34,0.0,0.50,528.0,...,2.4,18.669,7.778,0.800,75.0,28.35,1 oz,17.0,1 cubic inch,cheese blue
1,1006,48.42,334,20.75,27.68,2.70,0.45,0.0,0.45,184.0,...,2.3,17.410,8.013,0.826,100.0,28.35,1 oz,144.0,"1 cup, sliced",cheese brie
2,1007,51.80,300,19.80,24.26,3.68,0.46,0.0,0.46,388.0,...,2.0,15.259,7.023,0.724,72.0,28.35,1 oz,246.0,1 cup,cheese camembert
3,1009,37.02,404,22.87,33.31,3.71,3.09,0.0,0.48,710.0,...,2.4,18.867,9.246,1.421,99.0,132.00,"1 cup, diced",244.0,"1 cup, melted",cheese cheddar
4,1015,81.24,81,10.45,2.27,1.27,4.76,0.0,4.00,111.0,...,0.0,1.235,0.516,0.083,12.0,113.00,4 oz,226.0,"1 cup, (not packed)",cheese cottage lowfat 2%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,43392,69.50,115,2.70,0.20,1.99,25.61,1.5,17.16,18.0,...,0.0,0.046,0.005,0.089,0.0,NaN,NaN,NaN,NaN,hearts of palm
540,43572,2.80,429,12.60,9.50,1.71,73.39,14.2,0.54,11.0,...,15.7,1.415,4.085,3.572,0.0,28.35,1 oz,NaN,NaN,popcorn microwave lofat&na
541,43598,21.70,688,0.00,77.80,0.40,0.30,0.0,0.30,7.0,...,24.7,10.784,18.026,45.539,0.0,15.00,1 tbsp,239.0,1 cup,mayonnaise dressing
542,48052,8.20,370,75.16,1.85,1.00,13.79,0.6,0.00,142.0,...,0.0,0.272,0.156,0.810,0.0,NaN,NaN,NaN,NaN,vital wheat gluten


## Get valid recipes

In [375]:
valid_recipe_names = list(valid.keys())

recipes_csv = recipes[recipes['name'].isin(valid_recipe_names)].drop_duplicates(subset=['name'], keep='first').reset_index(drop=True)

recipes_csv

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,better than sex strawberries,42198,1460,41531,2002-10-03,"['weeknight', 'time-to-make', 'course', 'main-...","[734.1, 66.0, 199.0, 10.0, 10.0, 117.0, 28.0]",8,['crush vanilla wafers into fine crumbs and li...,simple but sexy. this was in my local newspape...,"[vanilla wafers, butter, powdered sugar, eggs,...",7
1,chinese chop suey,8559,70,4481,2001-01-27,"['weeknight', 'time-to-make', 'course', 'main-...","[395.4, 31.0, 20.0, 29.0, 51.0, 33.0, 8.0]",8,"['brown ground meat and onion in a large pot',...",easy one-pot dinner.,"[celery, onion, ground pork, soy sauce, beef b...",7
2,fried potatoes,37073,40,1533,2002-08-13,"['60-minutes-or-less', 'time-to-make', 'course...","[132.6, 8.0, 4.0, 3.0, 4.0, 5.0, 6.0]",14,"['preheat oven to 400 degrees', 'cut the potat...","my husband made these up last week, very tasty...","[red potatoes, margarine, rosemary]",3
3,momma s special marinade,30131,10,41480,2002-06-03,"['15-minutes-or-less', 'time-to-make', 'course...","[199.2, 31.0, 6.0, 10.0, 1.0, 13.0, 1.0]",4,['depending on how much meat i have i usually ...,we usually use with chicken. my neices ask for...,"[lemon juice, oil, worcestershire sauce, basil...",7
4,munch without guilt tomatoes,30300,10,6164,2002-06-04,"['15-minutes-or-less', 'time-to-make', 'course...","[3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",3,['put a slice of tomato on each biscuit / crac...,anytime munchies...another sweat free preparat...,"[tomatoes, crackers, mayonnaise, black pepper]",4
...,...,...,...,...,...,...,...,...,...,...,...,...
1406,bed and breakfast rolls,223742,30,129958,2007-04-20,"['30-minutes-or-less', 'time-to-make', 'course...","[105.4, 8.0, 22.0, 5.0, 3.0, 12.0, 3.0]",12,"['cream together cream cheese , mayonnaise and...","this recipe comes from my ""no time to cook"" co...","[cream cheese, mayonnaise, milk, white bread, ...",7
1407,bedouin fresh date sweet rangina gluten free,405481,17,593927,2009-12-29,"['30-minutes-or-less', 'time-to-make', 'course...","[725.5, 80.0, 126.0, 19.0, 7.0, 164.0, 21.0]",8,['remove pits from the dates and arrange on sm...,a simple delicious gulf arabian sweet. perfect...,"[dates, butter, rice flour, cardamom]",4
1408,beef cabbage alfredo,82816,20,122616,2004-02-02,"['30-minutes-or-less', 'time-to-make', 'course...","[619.9, 75.0, 22.0, 16.0, 71.0, 125.0, 3.0]",9,"['cook and drain beef', 'season with garlic , ...",NaN,"[ground beef, cabbage, butter, heavy whipping ...",6
1409,beef chimichurri sauce,307536,26,386585,2008-06-05,"['30-minutes-or-less', 'time-to-make', 'course...","[134.5, 20.0, 0.0, 12.0, 1.0, 9.0, 0.0]",13,"['in a food processor , combine parsley , shal...",summer is the perfect time to make this argent...,"[fresh parsley leaves, shallot, garlic cloves,...",8


## Make recipe-ingredient relations

In [376]:
def find_recipe_id(recipe_name):
  return recipes_csv[recipes_csv['name'] == recipe_name]['id'].item()

def find_ingredient_id(ingredient_name):
  return ingredients_csv[ingredients_csv['Cleaned_Desc'] == ingredient_name]["NDB_No"].item()

def create_relationship(recipe_name, ingredients):
  recipe_id = find_recipe_id(recipe_name)
  ingred_rows = [{"recipe_id": recipe_id, "ingredient_id": find_ingredient_id(value), "variant": key} for key, value in ingredients.items()]
  return ingred_rows

relation_temp = [create_relationship(recipe_name, ingredients) for recipe_name, ingredients in valid.items()]
ingredient_in_csv = pd.DataFrame([row for rows in relation_temp for row in rows])

ingredient_in_csv

,recipe_id,ingredient_id,variant
0,42198,18609,vanilla wafers
1,42198,1145,unsalted butter
2,42198,19336,powdered sugar
3,42198,1123,whole egg
4,42198,1053,heavy whipping cream
...,...,...,...
8783,182042,11124,carrots
8784,182042,2030,black pepper
8785,182042,2047,salt
8786,182042,5311,chicken broth


## Interactions

In [377]:
interactions = pd.read_csv('./raw_data/RAW_interactions.csv')
interactions_csv = interactions[interactions['recipe_id'].isin(recipes_csv['id'])].reset_index(drop=True)
interactions_csv

,user_id,recipe_id,date,rating,review
0,52476,181765,2006-08-15,5,We have been eating this bbq sauce for years! ...
1,37722,181765,2007-03-12,5,This is excellent and the same recipe I've alw...
2,135470,181765,2008-05-05,1,I did not care for this at all. All I could ta...
3,286566,352182,2011-02-16,4,We really liked the marinade but next time I w...
4,1802964648,424551,2014-07-25,4,When making this recipe be aware of the scovil...
...,...,...,...,...,...
9108,5060,39370,2002-11-24,5,"WOW!! Chris, made this for Dad and me for the ..."
9109,169430,190261,2007-01-14,5,"WOW, these are delicious! I used anise seed th..."
9110,58104,190261,2007-06-07,5,Tasty! A nice hint of anise and sweetness from...
9111,1197076,190261,2010-03-08,0,Delicious ! I tweeked the recipe a bit>>substi...


## Create User Data

In [378]:
user_ids = pd.concat([interactions_csv['user_id'], recipes_csv['contributor_id']], ignore_index=True).unique()
print("Unique users: ", len(user_ids))

Unique users:  6154


In [379]:
from faker import Faker

fake = Faker()

In [380]:
def generate_info():
  name = fake.first_name() + " " + fake.last_name()
  domain = fake.free_email_domain()
  split_name = name.lower().split(" ")
  rand = fake.random_int(min=0, max=100) % 7
  email = ""
  if rand == 1:
    email = split_name[0] + split_name[1] + "@" + domain
  elif rand == 2:
    email = split_name[0] + "." + split_name[1] + "@" + domain
  elif rand == 3:
    email = split_name[0][0] + split_name[1] + str(fake.random_int(min=1, max=100)) + "@" + domain
  elif rand == 4:
    email = split_name[0] + str(fake.random_int(min=1, max=100)) + "@" + domain
  elif rand == 5:
    email = split_name[1] + split_name[0] + "@" + domain
  elif rand == 6:
    email = split_name[1] + "." + split_name[0] + "@" + domain
  else:
    email = split_name[0] + "_" + split_name[1] + str(fake.random_int(min=1, max=100)) + "@" + domain
  return name, email
    

users_csv = pd.DataFrame([{"id": user_id, "name": name, "email": email} for user_id, (name, email) in zip(user_ids, [generate_info() for _ in user_ids])])

users_csv

,id,name,email
0,52476,Donald Richards,richardsdonald@gmail.com
1,37722,Christopher Hunt,christopher_hunt31@hotmail.com
2,135470,Alicia Walker,aliciawalker@gmail.com
3,286566,Paul Wright,paul_wright1@yahoo.com
4,1802964648,Dana Pierce,dana85@gmail.com
...,...,...,...
6149,1433633,Claire Wood,claire74@gmail.com
6150,2602981,Sherri Heath,sherri_heath49@hotmail.com
6151,121041,Samuel Hayes,samuel.hayes@hotmail.com
6152,122616,Hannah Stewart,stewart.hannah@yahoo.com


## Allergen Data

In [381]:
allergies_to_consider = ["nut", "lactose"]
allergens = pd.read_csv('./raw_data/RAW_allergens.csv')

allergens['Allergy'] = allergens['Allergy'].str.lower()
allergens['Food'] = allergens['Food'].str.lower()

allergens.dropna(subset=['Allergy'], inplace=True)

allergen_keywords = {allergen : allergens[allergens['Allergy'].str.contains(allergen)]["Food"].values for allergen in allergies_to_consider}

allergen_keywords

{'nut': array(['almond', 'chestnut', 'ginkgo nut', 'peanut', 'pecan', 'walnut'],
       dtype=object),
 'lactose': array(['butter', 'buttermilk', 'casein', 'cheese', 'cream', 'custard',
        'ice cream', 'lactose', 'milk', 'sour cream', 'whey', 'yogurt '],
       dtype=object)}

In [382]:
allergy_csv = pd.DataFrame([{"allergy": allergy, "allergy_id": hash(allergy) % 10000} for allergy in allergies_to_consider])

allergy_csv

,allergy,allergy_id
0,nut,7500
1,lactose,1416


In [383]:
allergens_list = []
for allergy in allergies_to_consider:
  keyword_list = np.append(allergen_keywords[allergy], allergy)
  allergens_list.extend([{"allergy_id": hash(allergy) % 10000, "ingredient_id": ingred_id} for ingred_id in ingredients_csv[ingredients_csv["Cleaned_Desc"].str.contains('|'.join(keyword_list), regex=True)]["NDB_No"]])

allergens_csv = pd.DataFrame(allergens_list)

allergens_csv

,allergy_id,ingredient_id
0,7500,2025
1,7500,4042
2,7500,4528
3,7500,4532
4,7500,11485
...,...,...
80,1416,18373
81,1416,19095
82,1416,22899
83,1416,42291


## Output

In [384]:
recipes_csv.to_csv('./cleaned_data/recipes.csv', index=False)
ingredients_csv.to_csv('./cleaned_data/ingredients.csv', index=False)
ingredient_in_csv.to_csv('./cleaned_data/ingredient_in.csv', index=False)
interactions_csv.to_csv('./cleaned_data/interactions.csv', index=False)
users_csv.to_csv('./cleaned_data/users.csv', index=False)
allergy_csv.to_csv('./cleaned_data/allergies.csv', index=False)
allergens_csv.to_csv('./cleaned_data/allergens.csv', index=False)